<a href="https://colab.research.google.com/github/Nigouz/3204-CourseWork2/blob/main/onehotencoding_3204_Coursework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import re

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
  %matplotlib inline
  plt.style.use('ggplot')
  # df = pd.read_json('/content/drive/MyDrive/3204 CourseWork 2/Gp15_dataset/raw_logs/filebeat_raw_Gp15_all.json', lines=True)
  df = pd.read_csv("combinedtraining_dataset.csv")
  #df = pd.read_csv("smalldata.csv")

  print(f"[*] Shape of dataset: {df.shape}")
  print(df.head())
  #print(df.tail())
  print("\n")

[*] Shape of dataset: (98411, 14)
     @timestamp         src_ip src_port      dst_ip dst_port Protocol  \
0  1.666574e+09  192.168.0.102     None   10.0.2.15     None    HTTPS   
1  1.666574e+09  192.168.0.102     None  172.17.0.1     None    HTTPS   
2  1.666574e+09  192.168.0.102     None  172.18.0.1     None    HTTPS   
3  1.666574e+09  192.168.0.102     None   10.0.2.15     None    HTTPS   
4  1.666574e+09  192.168.0.102     None  172.17.0.1     None    HTTPS   

  http_request_method http_request_referrer http_response_code url_path  \
0                 GET                  None                200        /   
1                 GET                  None                200        /   
2                 GET                  None                200        /   
3                 GET                  None                200        /   
4                 GET                  None                200        /   

  user_agent_original sql_method sql_query         category  
0             

In [ ]:
x = df.iloc[:, :13].values
y = df["category"].values
features = df.columns.values[:-1]

In [ ]:
from sklearn import metrics
scores = {}
scores_list= []

def scoring_metrics(y_test, y_pred, model):
    accuracy = metrics.accuracy_score(y_test, y_pred)
    scores[model] = accuracy
    scores_list.append(accuracy)
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

    print(f"\n[*] Model: {model}")
    print("[*] [green bold]True Positive:[/green bold] {}".format(confusion_matrix[0][0]))
    print("[*] [red bold]False Positive:[/red bold] {}".format(confusion_matrix[1][0]))
    print("[*] [red bold]False Negative:[/red bold] {}".format(confusion_matrix[0][1]))
    print("[*] [green bold]True Negative:[/green bold] {}".format(confusion_matrix[1][1]))
    print("[*] Precision: {:.3f}%".format(metrics.precision_score(y_test, y_pred,pos_label='port_scan')))
    print("[*] Recall: {:.3f}%".format(metrics.recall_score(y_test, y_pred,pos_label='port_scan')))

    print("[*] Accuracy: {:.3f}%".format(accuracy))
    print("[*] F1_score: {:.3f}%".format(metrics.f1_score(y_test, y_pred,pos_label='port_scan')))


# KNN

In [ ]:
"""from sklearn import preprocessing

df.replace(to_replace=["None"], value=np.nan, inplace=True)
clean_df = df.fillna(0)
clean_x = clean_df.iloc[:, :13].values
clean_y = clean_df["category"].values
features = df.columns.values[:-1]

for label in clean_df.columns:
    for index, rows in clean_df.iterrows():
        new_ip = ""
        ip = str(rows[label])
        if re.search("\d+\.\d+\.\d+\.\d+", ip):
            octets = ip.split(".")
            for octet in octets:
                octet = octet.rjust(3,"0")
                new_ip += octet
            clean_df[label][index] = new_ip

#to label encode our data
def labelencode(data):
  le = preprocessing.LabelEncoder()
  nonull = np.array(data.dropna())
  impute_reshape = nonull.reshape(-1,1)
  impute_encode = le.fit_transform(impute_reshape)
  data.loc [data.notnull()]= np.squeeze(impute_encode)
  print(data)


labelencode(clean_df["Protocol"])
clean_x = clean_df.iloc[:, :13].values
clean_y = clean_df["category"].values
features = df.columns.values[:-1]

(clean_df.iloc[:,:13]).info()
"""

In [ ]:
from sklearn.preprocessing import OneHotEncoder
df.replace(to_replace=["None"], value=np.nan, inplace=True)
clean_df = df.fillna(str(0)) #uhm i dont knowhow to explain this, but please lmk another alternative to fix this cause :") try to run it without str and you will know what i mean"
clean_x = clean_df.iloc[:, :13].values
clean_y = clean_df["category"].values
features = df.columns.values[:-1]

for label in clean_df.columns:
    for index, rows in clean_df.iterrows():
        new_ip = ""
        ip = str(rows[label])
        if re.search("\d+\.\d+\.\d+\.\d+", ip):
            octets = ip.split(".")
            for octet in octets:
                octet = octet.rjust(3,"0")
                new_ip += octet
            clean_df[label][index] = new_ip

"""
#print(df.dtypes) 
#print(df["Protocol"].unique())  
enc = OneHotEncoder(handle_unknown='ignore') #ignore any unknown categories
npdata = np.array(clean_df[["Protocol","http_request_method"]])
reshapeddata = npdata.reshape(-1,1)

onehotencode(clean_df["Protocol"])
clean_x = clean_df.iloc[:, :13].values
clean_y = clean_df["category"].values
features = df.columns.values[:-1]

(clean_df.iloc[:,:13]).info()"""

'onehotencode(clean_df["Protocol"])\nclean_x = clean_df.iloc[:, :13].values\nclean_y = clean_df["category"].values\nfeatures = df.columns.values[:-1]\n\n(clean_df.iloc[:,:13]).info()'

In [ ]:
from sklearn.compose import make_column_transformer
clean_x = clean_df.iloc[:, :13] #honestly don't know if this is correct lmaoooo
#this is to iterate through the columns and convert the strings to float (passthrough means for those columns we didnt specify, leave it as it is)
column_trans = make_column_transformer((OneHotEncoder(sparse=False), ['Protocol', 'http_request_method', 'url_path', 'user_agent_original']),remainder='passthrough')
test = column_trans.fit_transform(clean_x) #this is technically our clean_x after the column transfer 

In [ ]:
"""feature_array= enc.fit_transform(reshapeddata)
print(feature_array)
enc.categories_
feature_labels = enc.categories_
feature_labels =np.array(feature_labels).ravel()
print (feature_labels)"""

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train, x_test, y_train, y_test = train_test_split(test, clean_y, test_size=0.3)

In [ ]:
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x_train, y_train)
pickle.dump(knn, open(f"knn{k}.sav", "wb"))

y_pred = knn.predict(x_test)
scoring_metrics(y_test, y_pred, f"knn {k}")


[*] Model: knn 5
[*] [green bold]True Positive:[/green bold] 68
[*] [red bold]False Positive:[/red bold] 0
[*] [red bold]False Negative:[/red bold] 1
[*] [green bold]True Negative:[/green bold] 8
[*] Precision: 1.000%
[*] Recall: 0.986%
[*] Accuracy: 0.987%
[*] F1_score: 0.993%


# SVM

In [ ]:
from sklearn.svm import SVC

# instantiate the model
svm = SVC(kernel='linear', C=1.0, random_state=12)
#fit the model
svm.fit(X_train, y_train)

In [ ]:
#predicting the target value from the model for the samples
y_test_svm = svm.predict(X_test)
y_train_svm = svm.predict(X_train)

In [ ]:
#computing the accuracy of the model performance
acc_train_svm = accuracy_score(y_train,y_train_svm)
acc_test_svm = accuracy_score(y_test,y_test_svm)

# Isolation Forest

In [ ]:
df = pd.read_json('logs.json', lines=True)
df = df[]